In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [2]:
products = pd.read_csv('/Users/carolinalapid/Desktop/TFM/products_df.csv',index_col=0)

sd = pd.read_csv('/Users/carolinalapid/Desktop/TFM/sociodemographic_df.csv',index_col=0)

cm = pd.read_csv('/Users/carolinalapid/Desktop/TFM/commercial_activity_df.csv',index_col=0)

# Descripción de las variables
### 1. Products
#### Cuentas

em_account_p: Cuenta easyMoney+

em_account_pp: Cuenta easyMoney++

em_acount: Cuenta easyMoney

emc_account: Cuenta easyMoney Crypto

#### payroll: Domiciliaciones

payroll_account: Cuenta bonificada por domiciliaciones

debit_card: Tarjetas de débito


#### Productos de ahorro e inversión

funds: Fondos de inversión

long_term_deposit: Depósitos a largo plazo

pension_plan: Plan de pensiones

securities: Valores

short_term_deposit: Short-term deposits


#### Productos de financiación

credit_card: Tarjetas de crédito

loans: Préstamos

mortgage: hipoteca


### 2. Commercial_activity

entry_channel: Canal de captación del cliente

entry_date: Fecha en la que realizó la primera contratación a través de easyMoney

segment: Segmento comercial del cliente

active_customer: Indicador de actividad del cliente en nuestra aplicación


### 3. Sociodemographic

age: Edad del cliente

country_id: País de residencia del cliente

deceased: Deceased index. N/S

gender: Sexo del cliente

region_code: Provincia de residencia del cliente (para ES)

salary: Ingresos brutos de la unidad familiar

In [3]:
df1= pd.merge(products,cm, how="inner",on=['pk_cid','pk_partition' ])
df=pd.merge(df1,sd, how="inner",on=['pk_cid','pk_partition'])

In [4]:
df.shape

(5962924, 27)

In [5]:
df.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'entry_date',
       'entry_channel', 'active_customer', 'segment', 'country_id',
       'region_code', 'gender', 'age', 'deceased', 'salary'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
 17  entry_date          object 
 18  entry_channel       object 
 19  active_customer     float64
 20  segment             object 
 21  country_id          object 
 22  region_code         float64
 23  gender              object 
 24  age                 int6

In [7]:
df.head(3).T

,0,1,2
pk_cid,1375586,1050611,1050612
pk_partition,2018-01-28,2018-01-28,2018-01-28
short_term_deposit,0,0,0
loans,0,0,0
mortgage,0,0,0
funds,0,0,0
securities,0,0,0
long_term_deposit,0,0,0
em_account_pp,0,0,0
credit_card,0,0,0


In [8]:
dupl_rows = df.duplicated().sum()
print("The number of duplicated rows is:", dupl_rows)

The number of duplicated rows is: 0


In [9]:
df = df.sort_values(by=['pk_cid', 'pk_partition'])

In [10]:
df.columns[df.isnull().any()]

Index(['payroll', 'pension_plan', 'entry_channel', 'segment', 'region_code',
       'gender', 'salary'],
      dtype='object')

In [11]:
## Tratamiento de nulos: Empezamos estudiando las categóricas , para eso dividimos las columnas en función de sus dtypes.

In [12]:
# Identificamos las variables numéricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Identificamos las variables fechas
data_cols = df.select_dtypes(include=['datetime64[ns]']).columns

# Identificamos las variables categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Identificamos las variables booleanas
boolean_cols = df.select_dtypes(include=['bool']).columns

In [13]:
categorical_cols=list(categorical_cols)
data_cols=list(data_cols)
numeric_cols = list(numeric_cols)
boolean_cols = list(boolean_cols)

In [14]:
df.isnull().sum()

pk_cid                      0
pk_partition                0
short_term_deposit          0
loans                       0
mortgage                    0
funds                       0
securities                  0
long_term_deposit           0
em_account_pp               0
credit_card                 0
payroll                    61
pension_plan               61
payroll_account             0
emc_account                 0
debit_card                  0
em_account_p                0
em_acount                   0
entry_date                  0
entry_channel          133033
active_customer             0
segment                133944
country_id                  0
region_code              2264
gender                     25
age                         0
deceased                    0
salary                1512103
dtype: int64

In [15]:
df.value_counts("payroll")

payroll
0.0    5757076
1.0     205787
dtype: int64

In [16]:
products.value_counts("pension_plan")

pension_plan
0.0    5745061
1.0     217802
dtype: int64

In [17]:
df['payroll'].fillna(0.0,inplace=True)
df['pension_plan'].fillna(0.0,inplace=True)

In [18]:
df["entry_channel"].value_counts()

KHE    3113947
KFC     890620
KHQ     590280
KAT     416084
KHK     230197
        ...   
KEJ          8
KHS          5
KFP          2
KDA          2
KDS          1
Name: entry_channel, Length: 68, dtype: int64

In [19]:
df['entry_date'].fillna("sin info",inplace=True)

In [20]:
#Ademas de quitarle los nulos, vamos a agruparla por los principales canales

In [21]:
lista_channels= ["KHE","KFC","KHQ","KAT","KHK"]

In [22]:
lista_channels_menosfrecuente= df["entry_channel"].value_counts().tail(63).index.tolist()

In [23]:
df.loc[df["entry_channel"].isin(lista_channels_menosfrecuente), "entry_channel"]="OTHERS"

In [24]:
df['entry_channel'].value_counts()
df['entry_channel'].fillna("OTHERS",inplace=True)

In [25]:
df["segment"].value_counts()

03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
01 - TOP                97939
Name: segment, dtype: int64

In [26]:
df['segment'].fillna('04 - SIN CATEGORIA', inplace= True)

In [27]:
df["gender"].value_counts()

H    3087502
V    2875397
Name: gender, dtype: int64

In [28]:
df['gender'].fillna('V', inplace= True)

In [29]:
salario_segmento =df.groupby('segment')['salary'].mean()
salario_segmento

segment
01 - TOP              148448.090709
02 - PARTICULARES     115887.830537
03 - UNIVERSITARIO    114978.212778
04 - SIN CATEGORIA    113070.572915
Name: salary, dtype: float64

En este caso vemos que particulares y universitarios presentan salarios similares, mientras que los que tienen top presentan una media salarial mayor

La forma con la que imputamos los nulos es en base a la agrupacion que tienen con segment, imputamos los nulos de salary teniendo en cuenta cada uno de los niveles

In [30]:
df['salary'] = df.groupby('segment')['salary'].transform(lambda x: x.fillna(x.mean()))
df['salary'].isnull().sum().sum()

0

In [31]:
df["region_code"].value_counts()

28.0    1185757
8.0      600362
46.0     358142
30.0     296856
41.0     290447
15.0     254891
29.0     187911
36.0     175850
3.0      171358
11.0     160894
33.0     134562
50.0     132983
6.0      122587
35.0     121982
47.0     116948
18.0     102789
45.0     102328
37.0      92538
10.0      83283
14.0      75405
2.0       72071
21.0      72037
13.0      70092
39.0      69589
12.0      68709
7.0       62436
32.0      54639
27.0      52783
43.0      51782
17.0      50749
9.0       47746
25.0      46887
16.0      36094
24.0      35426
48.0      35256
26.0      34790
4.0       32680
31.0      31241
49.0      30965
23.0      30955
38.0      29001
34.0      27545
19.0      26762
40.0      21658
5.0       21300
22.0      21112
20.0      18833
1.0       12999
44.0      12616
42.0       8500
51.0       2896
52.0       2638
Name: region_code, dtype: int64

In [32]:
provincias={1:'Alava',
2:'Albacete',
3:'Alicante',
4:'Almeria',
5:'Avila',
6:'Badajoz',
7:'Baleares',
8:'Barcelona',
9:'Burgos',
10:'Caceres',
11:'Cadiz',
12:'Castellon',
13:'Ciudad Real',
14:'Cordoba',
15:'La Coruna',
16:'Cuenca',
17:'Gerona',
18:'Granada',
19:'Guadalajara',
20:'Guipuzcoa',
21:'Huelva',
22:'Huesca',
23:'Jaen',
24:'Leon',
25:'Lerida',
26:'La Rioja',
27:'Lugo',
28:'Madrid',
29:'Malaga',
30:'Murcia',
31:'Navarra',
32:'Orense',
33:'Asturias',
34:'Palencia',
35:'Las Palmas',
36:'Pontevedra',
37:'Salamanca',
38:'Santa Cruz de Tenerife',
39:'Cantabria',
40:'Segovia',
41:'Sevilla',
42:'Soria',
43:'Tarragona',
44:'Teruel',
45:'Toledo',
46:'Valencia',
47:'Valladolid',
48:'Vizcaya',
49:'Zamora',
50:'Zaragoza',
51:'Ceuta',
52:'Melilla'}

In [33]:
df['provincia']=df['region_code'].map(provincias)

In [34]:
df['provincia'].mode().iloc[0]

'Madrid'

In [35]:
p_mode= df['region_code'].mode().iloc[0]

In [36]:
df['region_code'].fillna(p_mode, inplace=True)

In [37]:
df.isnull().sum()

pk_cid                   0
pk_partition             0
short_term_deposit       0
loans                    0
mortgage                 0
funds                    0
securities               0
long_term_deposit        0
em_account_pp            0
credit_card              0
payroll                  0
pension_plan             0
payroll_account          0
emc_account              0
debit_card               0
em_account_p             0
em_acount                0
entry_date               0
entry_channel            0
active_customer          0
segment                  0
country_id               0
region_code              0
gender                   0
age                      0
deceased                 0
salary                   0
provincia             2264
dtype: int64

In [38]:
#Ahora que no tenemos nulos, pasaremos a mirar cuales clientes son nuevos:

In [39]:
#corregimos los 29 de febrero y los reemplazamos por 28 de febrero y transformamos el tipo de variable a datetype

In [40]:
df.loc[ (df['entry_date']=='2019-02-29'), 
              'entry_date']='2019-02-28'
df.loc[ (df['entry_date']=='2015-02-29'), 
              'entry_date']='2015-02-28'

In [41]:
for i in ["pk_partition","entry_date"]:
    df[i]=pd.to_datetime(df[i], format='%Y-%m-%d')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 28 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             float64       
 11  pension_plan        float64       
 12  payroll_account     int64         
 13  emc_account         int64         
 14  debit_card          int64         
 15  em_account_p        int64         
 16  em_acount           int64         
 17  entry_date          datetime64[ns]
 18  entry_channel       object        
 19  active_customer     float64       
 

In [43]:
#Asigna a una nueva columna llamada 'meses_alta' en el DataFrame. Cada fila de esta columna contiene el número de meses que han transcurrido desde la fecha de entrada hasta la fecha de referencia para ese registro en particular.

In [44]:
df['meses_alta']=(df['pk_partition']- df['entry_date'])/np.timedelta64(1,'M')

In [45]:
df['New_Client']=((df['meses_alta'] < 1) & 
                          (df['meses_alta'] > 0)).astype(int)

In [46]:
df.head(3)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,segment,country_id,region_code,gender,age,deceased,salary,provincia,meses_alta,New_Client
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,04 - SIN CATEGORIA,ES,28.0,H,59,N,113070.572915,Madrid,0.000000,0
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,02 - PARTICULARES,ES,28.0,H,59,N,115887.830537,Madrid,1.018501,0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,04 - SIN CATEGORIA,ES,28.0,H,62,N,113070.572915,Madrid,0.295694,1


In [47]:
#exportamos el df limpio y con la informacion de new clients

In [48]:
df_clean = df.copy()

In [49]:
df_clean.to_csv("/Users/carolinalapid/Desktop/TFM/df_clean.csv")

Creamos un DF nuevo para analizar las ventas del ultimo mes y poder visualizar cuantos productos hemos vendido este mes o si son los clientes nuevos quiénes los compran

In [50]:
df_1= df[['entry_date','pk_partition', 'segment','short_term_deposit','loans','mortgage','funds','securities','long_term_deposit','em_account_pp','credit_card','payroll','pension_plan','payroll_account','emc_account',
          'debit_card', 'em_account_p','em_acount' ]]
df_1.head()

,entry_date,pk_partition,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
1479563,2018-07-28,2018-07-28,04 - SIN CATEGORIA,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2168122,2018-07-28,2018-08-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
2962973,2018-11-19,2018-11-28,04 - SIN CATEGORIA,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
3628236,2018-11-19,2018-12-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4028169,2018-11-19,2019-01-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0


In [51]:
df_1.groupby('entry_date').sum().reset_index()
ventas = df_1.groupby('entry_date').sum().reset_index()

In [52]:
ventas.head()

,entry_date,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,2015-01-01,0,0,0,0,0,0,0,17,0.0,0.0,0,0,0,0,0
1,2015-01-02,0,0,0,0,16,38,0,195,190.0,204.0,323,674,438,0,1870
2,2015-01-03,0,0,0,41,53,126,0,100,120.0,120.0,239,502,387,0,899
3,2015-01-04,0,0,0,0,33,44,0,125,102.0,116.0,176,558,289,0,577
4,2015-01-05,0,0,0,0,17,33,0,64,87.0,87.0,152,381,222,0,313


In [53]:
#ahora que tenemos las ventas de todos los productos por fecha las exportamos en un csv

ventas.to_csv("/Users/carolinalapid/Desktop/TFM/ventas.csv")

In [54]:
ventas1 = df_1.groupby('pk_partition').sum().reset_index()

In [55]:
ventas1.head()

,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,2018-01-28,883,19,15,786,932,4884,0,3325,8145.0,8835.0,13478,15320,24696,2,215293
1,2018-02-28,1343,19,15,872,965,4956,0,3282,8889.0,9496.0,14246,15827,25448,2,217098
2,2018-03-28,1664,23,17,969,965,4965,0,3511,9735.0,9882.0,14989,16428,27046,2,218683
3,2018-04-28,1740,24,17,1084,983,5107,0,3725,9905.0,10560.0,15825,16947,27764,2,219816
4,2018-05-28,1455,27,18,1145,1012,5356,0,3797,9994.0,10148.0,16697,17569,27911,2,221291


In [56]:
ventas1.to_csv("/Users/carolinalapid/Desktop/TFM/ventas1.csv")

In [57]:
#Creamos una variable de total_products

In [58]:
productos_easymoney= ['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [59]:
df_prod_total = df_1.melt(id_vars=['pk_partition'],
              value_vars = productos_easymoney,
              var_name ='Product',
              value_name ='Count')

In [60]:
df_prod_total.tail()

,pk_partition,Product,Count
83480931,2019-05-28,em_acount,0.0
83480932,2019-05-28,em_acount,0.0
83480933,2019-05-28,em_acount,0.0
83480934,2019-05-28,em_acount,0.0
83480935,2019-05-28,em_acount,0.0


In [61]:
df_prod_total =df_prod_total.groupby(['pk_partition']).agg({'Count':np.sum}).reset_index(drop=False)

In [62]:
df_prod_total.tail()

,pk_partition,Count
12,2019-01-28,411164.0
13,2019-02-28,421404.0
14,2019-03-28,428651.0
15,2019-04-28,432663.0
16,2019-05-28,438696.0


In [63]:
df_prod_total.rename(columns={'Count':'total_productos'}, inplace=True)

In [64]:
len(df_1),len(df_prod_total)

(5962924, 17)

In [65]:
df_final=pd.merge(df_1,df_prod_total, how="inner",on=['pk_partition'])


In [66]:
df_final.tail()

,entry_date,pk_partition,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,total_productos
5962919,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,321043.0
5962920,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962921,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962922,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962923,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0


In [67]:
df_final['total_productos'].value_counts().to_frame()

,total_productos
438696.0,442995
432663.0,439627
428651.0,436183
421404.0,431727
411164.0,426875
411614.0,422481
401801.0,416387
389961.0,402300
367786.0,375323
347677.0,352922


In [68]:
df_final.to_csv("/Users/carolinalapid/Desktop/TFM/df_final.csv")

In [ ]:
#volvemos a usar el DF_CLEAN para trabajar sobre altas y productos en el dataset

In [111]:
#ahora vamos a crear la variable para saber si un cliente esta activo o no. Si el cliente no tiene ningun producto, no esta activo

In [112]:
df_clean['Cliente_activo']=((df_clean['loans']==0) &
                        (df_clean['mortgage']==0) &
                        (df_clean['funds']==0) &
                        (df_clean['securities']==0) &
                        (df_clean['long_term_deposit']==0) &
                        (df_clean['em_account_pp']==0) &
                        (df_clean['credit_card']==0) &
                        (df_clean['payroll']==0) &
                        (df_clean['pension_plan']==0) &
                        (df_clean['payroll_account']==0) &
                        (df_clean['emc_account']==0) &
                        (df_clean['debit_card']==0) &
                        (df_clean['em_account_p']==0) &
                        (df_clean['em_acount']==0)).astype(int)

In [113]:
#Ahora lo invertimos
df_clean['Cliente_activo']=(df_clean['Cliente_activo']!=1).astype(int)

In [114]:
df_clean.head(3)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,country_id,region_code,gender,age,deceased,salary,provincia,meses_alta,New_Client,Cliente_activo
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,ES,28.0,H,59,N,113070.572915,Madrid,0.000000,0,1
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,ES,28.0,H,59,N,115887.830537,Madrid,1.018501,0,0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,ES,28.0,H,62,N,113070.572915,Madrid,0.295694,1,0


In [116]:
df_clean['meses_alta']=(df_clean['pk_partition']- df_clean['entry_date'])/np.timedelta64(1,'M')

In [117]:
df_clean['Cliente_nuevo']=((df_clean['meses_alta'] < 1) & 
                          (df_clean['meses_alta'] > 0)).astype(int)

In [118]:
#juntamos en el mismo dataset la variable cliente nuevo (que habia quedado en df_clean)

In [132]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 47 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   pk_cid                 int64         
 1   pk_partition           datetime64[ns]
 2   short_term_deposit     int64         
 3   loans                  int64         
 4   mortgage               int64         
 5   funds                  int64         
 6   securities             int64         
 7   long_term_deposit      int64         
 8   em_account_pp          int64         
 9   credit_card            int64         
 10  payroll                float64       
 11  pension_plan           float64       
 12  payroll_account        int64         
 13  emc_account            int64         
 14  debit_card             int64         
 15  em_account_p           int64         
 16  em_acount              int64         
 17  entry_date             datetime64[ns]
 18  entry_channel   

In [119]:
df_clean.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,region_code,gender,age,deceased,salary,provincia,meses_alta,New_Client,Cliente_activo,Cliente_nuevo
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,28.0,H,59,N,113070.572915,Madrid,0.000000,0,1,0
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,28.0,H,59,N,115887.830537,Madrid,1.018501,0,0,0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,28.0,H,62,N,113070.572915,Madrid,0.295694,1,0,1
3628236,16063,2018-12-28,0,0,0,0,0,0,0,0,...,28.0,H,62,N,115887.830537,Madrid,1.281340,0,0,0
4028169,16063,2019-01-28,0,0,0,0,0,0,0,0,...,28.0,H,62,N,115887.830537,Madrid,2.299842,0,0,0


In [121]:
df_clean.to_csv("/Users/carolinalapid/Desktop/TFM/df_TFM.csv")

In [81]:
#creamos una lista de los productos easymoney

In [122]:
productos_easymoney=['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [123]:
def calcula_diferencias_mensuales (dataset, variable):
    dataset[variable+'_pm']  = dataset.groupby('pk_cid')[variable].shift(1)
    dataset['dif_'+variable] = dataset[variable] - dataset[variable+'_pm']
    dataset['dif_'+variable]  = dataset.groupby('pk_cid')[variable].diff()
    dataset.drop(variable+'_pm',axis=1,inplace=True)

In [125]:
df_clean.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,dif_long_term_deposit,dif_em_account_pp,dif_credit_card,dif_payroll,dif_pension_plan,dif_payroll_account,dif_emc_account,dif_debit_card,em_account_p_pm,dif_em_account_p
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3628236,16063,2018-12-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4028169,16063,2019-01-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
df_clean['em_acount'+'_pm']  = df_clean.groupby('pk_cid')['em_acount'].shift(1)

In [141]:
df_clean['dif_'+'em_acount'] = df_clean['em_acount'] - df_clean['em_acount'+'_pm']

In [142]:
df_clean['dif_'+'em_acount']  = df_clean.groupby('pk_cid')['em_acount'].diff()

In [143]:
df_clean.drop('em_acount'+'_pm',axis=1,inplace=True)

In [144]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 45 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   pk_cid                 int64         
 1   pk_partition           datetime64[ns]
 2   short_term_deposit     int64         
 3   loans                  int64         
 4   mortgage               int64         
 5   funds                  int64         
 6   securities             int64         
 7   long_term_deposit      int64         
 8   em_account_pp          int64         
 9   credit_card            int64         
 10  payroll                float64       
 11  pension_plan           float64       
 12  payroll_account        int64         
 13  emc_account            int64         
 14  debit_card             int64         
 15  em_account_p           int64         
 16  em_acount              int64         
 17  entry_date             datetime64[ns]
 18  entry_channel   

In [135]:
df_clean.drop('long_term_deposit_pm', axis=1, inplace=True)

In [136]:
df_clean.drop('New_Client', axis=1, inplace=True)

In [145]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 45 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   pk_cid                 int64         
 1   pk_partition           datetime64[ns]
 2   short_term_deposit     int64         
 3   loans                  int64         
 4   mortgage               int64         
 5   funds                  int64         
 6   securities             int64         
 7   long_term_deposit      int64         
 8   em_account_pp          int64         
 9   credit_card            int64         
 10  payroll                float64       
 11  pension_plan           float64       
 12  payroll_account        int64         
 13  emc_account            int64         
 14  debit_card             int64         
 15  em_account_p           int64         
 16  em_acount              int64         
 17  entry_date             datetime64[ns]
 18  entry_channel   

In [146]:
df_clean.head(3)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,dif_long_term_deposit,dif_em_account_pp,dif_credit_card,dif_payroll,dif_pension_plan,dif_payroll_account,dif_emc_account,dif_debit_card,dif_em_account_p,dif_em_acount
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
#quitamos los nulos de las variables diff

In [148]:
dif_productos_easymoney=['dif_loans',
 'dif_mortgage',
 'dif_funds',
 'dif_securities',
 'dif_long_term_deposit',
 'dif_em_account_pp',
 'dif_credit_card',
 'dif_payroll',
 'dif_pension_plan',
 'dif_payroll_account',
 'dif_emc_account',
 'dif_debit_card',
 'dif_em_account_p',
 'dif_em_acount']

In [150]:
for x in dif_productos_easymoney:
    df_clean[x].fillna(0,inplace=True)

In [151]:
df_clean.head(2)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,dif_long_term_deposit,dif_em_account_pp,dif_credit_card,dif_payroll,dif_pension_plan,dif_payroll_account,dif_emc_account,dif_debit_card,dif_em_account_p,dif_em_acount
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [152]:
DF = df_clean.copy()

In [153]:
#vamos a calcular las ventas mensuales por producto

In [154]:
dif_productos_easymoney=['dif_loans',
 'dif_mortgage',
 'dif_funds',
 'dif_securities',
 'dif_long_term_deposit',
 'dif_em_account_pp',
 'dif_credit_card',
 'dif_payroll',
 'dif_pension_plan',
 'dif_payroll_account',
 'dif_emc_account',
 'dif_debit_card',
 'dif_em_account_p',
 'dif_em_acount']

In [155]:
#vamos a hacer una tabla para ver la suma de productos por mes

In [156]:
altas = DF.pivot_table(index='pk_partition',
                      values=dif_productos_easymoney, aggfunc=[np.sum])

In [157]:
altas.head()

sum                                  \
             dif_credit_card dif_debit_card dif_em_account_p   
pk_partition                                                   
2018-01-28               0.0            0.0              0.0   
2018-02-28             -72.0          704.0              0.0   
2018-03-28             198.0         1509.0              0.0   
2018-04-28             228.0          665.0              0.0   
2018-05-28              49.0           98.0              0.0   

                                                                        \
             dif_em_account_pp dif_em_acount dif_emc_account dif_funds   
pk_partition                                                             
2018-01-28                 0.0           0.0             0.0       0.0   
2018-02-28                 0.0       -1481.0           493.0      85.0   
2018-03-28                 0.0       -1419.0           588.0      93.0   
2018-04-28                 0.0       -1430.0           504.0     111.0   
2018-05-28                 0.0       -1216.0           610.0      55.0   

                                                                       \
             dif_loans dif_long_term_deposit dif_mortgage dif_payroll   
pk_partition                                                            
2018-01-28         0.0                   0.0          0.0         0.0   
2018-02-28         0.0                  46.0          0.0       696.0   
2018-03-28         4.0                 -37.0          2.0       776.0   
2018-04-28         0.0                  96.0          0.0       116.0   
2018-05-28         3.0                 191.0          1.0        47.0   

                                                                  
             dif_payroll_account dif_pension_plan dif_securities  
pk_partition                                                      
2018-01-28                   0.0              0.0            0.0  
2018-02-28                 709.0            612.0           30.0  
2018-03-28                 691.0            317.0           -2.0  
2018-04-28                 774.0            619.0           15.0  
2018-05-28                 808.0           -454.0           23.0

In [160]:
#cambiamos nombres de columnas

In [161]:
altas.columns=['credit_card','debit_card',
                            'em_account_p',
                            'em_account_pp',
                             'em_acount',
                            'emc_account',
                            'funds',
                            'loans',
                            'long_term_deposit',
                            'mortgage',
                            'payroll',
                            'payroll_account',
                            'pension_plan',
                            'securities']

In [163]:
altas.head(17)

,credit_card,debit_card,em_account_p,em_account_pp,em_acount,emc_account,funds,loans,long_term_deposit,mortgage,payroll,payroll_account,pension_plan,securities
pk_partition,,,,,,,,,,,,,,
2018-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-02-28,-72.0,704.0,0.0,0.0,-1481.0,493.0,85.0,0.0,46.0,0.0,696.0,709.0,612.0,30.0
2018-03-28,198.0,1509.0,0.0,0.0,-1419.0,588.0,93.0,4.0,-37.0,2.0,776.0,691.0,317.0,-2.0
2018-04-28,228.0,665.0,0.0,0.0,-1430.0,504.0,111.0,0.0,96.0,0.0,116.0,774.0,619.0,15.0
2018-05-28,49.0,98.0,0.0,0.0,-1216.0,610.0,55.0,3.0,191.0,1.0,47.0,808.0,-454.0,23.0
2018-06-28,213.0,860.0,0.0,0.0,602.0,386.0,13.0,0.0,164.0,1.0,901.0,-1208.0,1433.0,0.0
2018-07-28,71.0,492.0,0.0,0.0,-2184.0,212.0,12.0,1.0,135.0,1.0,817.0,1299.0,1017.0,28.0
2018-08-28,67.0,-417.0,0.0,0.0,956.0,142.0,47.0,1.0,93.0,0.0,-537.0,1166.0,-685.0,78.0
2018-09-28,80.0,2407.0,0.0,0.0,-471.0,276.0,16.0,2.0,182.0,0.0,425.0,709.0,454.0,65.0


In [164]:
altas.reset_index(drop=False,inplace=True)

In [165]:
altas

,pk_partition,credit_card,debit_card,em_account_p,em_account_pp,em_acount,emc_account,funds,loans,long_term_deposit,mortgage,payroll,payroll_account,pension_plan,securities
0,2018-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-02-28,-72.0,704.0,0.0,0.0,-1481.0,493.0,85.0,0.0,46.0,0.0,696.0,709.0,612.0,30.0
2,2018-03-28,198.0,1509.0,0.0,0.0,-1419.0,588.0,93.0,4.0,-37.0,2.0,776.0,691.0,317.0,-2.0
3,2018-04-28,228.0,665.0,0.0,0.0,-1430.0,504.0,111.0,0.0,96.0,0.0,116.0,774.0,619.0,15.0
4,2018-05-28,49.0,98.0,0.0,0.0,-1216.0,610.0,55.0,3.0,191.0,1.0,47.0,808.0,-454.0,23.0
5,2018-06-28,213.0,860.0,0.0,0.0,602.0,386.0,13.0,0.0,164.0,1.0,901.0,-1208.0,1433.0,0.0
6,2018-07-28,71.0,492.0,0.0,0.0,-2184.0,212.0,12.0,1.0,135.0,1.0,817.0,1299.0,1017.0,28.0
7,2018-08-28,67.0,-417.0,0.0,0.0,956.0,142.0,47.0,1.0,93.0,0.0,-537.0,1166.0,-685.0,78.0
8,2018-09-28,80.0,2407.0,0.0,0.0,-471.0,276.0,16.0,2.0,182.0,0.0,425.0,709.0,454.0,65.0
9,2018-10-28,95.0,2641.0,0.0,0.0,625.0,216.0,10.0,2.0,157.0,-1.0,554.0,1042.0,548.0,144.0


In [167]:
altas=altas.melt(id_vars=['pk_partition'],
              value_vars=altas.columns[1:],
              var_name='Product',
              value_name='Count')

In [168]:
altas

,pk_partition,Product,Count
0,2018-01-28,credit_card,0.0
1,2018-02-28,credit_card,-72.0
2,2018-03-28,credit_card,198.0
3,2018-04-28,credit_card,228.0
4,2018-05-28,credit_card,49.0
...,...,...,...
233,2019-01-28,securities,231.0
234,2019-02-28,securities,87.0
235,2019-03-28,securities,20.0
236,2019-04-28,securities,10.0


In [169]:
altas.sort_values(by=['pk_partition','Product'],ascending=[True,False],inplace=True)

In [170]:
altas.to_csv("/Users/carolinalapid/Desktop/TFM/altas.csv")